In [1]:
import numpy as np
import os
import glob
import pandas as pd

# Image processing tools
import skimage
import skimage.io
import skimage.filters
import skimage.morphology

# import bokeh
# import bokeh.io #for some reason have to separately import bokeh.io
# bokeh.io.output_notebook()

# rename folders to change _# into a well name

In [2]:
def get_tuples_of_interest (dirpath):
    
    #get only terminal directories that will have images in them, walk also reports the parent directory its told to walk in
    #and we want to get rid of that
    tuples_of_interest = []
    for entry in list(os.walk(dirpath)):
        #if there are no subdirectories in this folder (meaning its a terminal directory that has image files in it)
        if not entry[1]:
            #tuples contain [0] the path for the directory, [1] any subdirectories in this one, [2] files in this directory
            tuples_of_interest.append(entry)
            
    return tuples_of_interest

In [3]:
directory = 'Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/'

tuples_of_interest = get_tuples_of_interest(directory)

In [4]:
len(tuples_of_interest)

216

In [5]:
tuples_of_interest[0][0].split('/')

['Z:',
 'Reed',
 'Projects',
 'micro_consortia',
 'DARPA_biocon',
 'Task 1.1',
 'A=B',
 '20190910 a=b manual dil degorig 1',
 'images',
 'a1 10to2_1']

In [6]:
#create non general dictionary to map '_#' suffixes on microscope picture folders to well names
#this will need to be updated when you have a new experiment with new well maps etc.

num = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
let = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
wells = [l+n for l in let for n in num]
suff = [str(x) for x in list(range(1,len(wells)+1))]

rename = {s:w for s,w in zip(suff, wells)}

In [9]:
directories = []

#this step needs to be updated when the length of the time string changes

for t in tuples_of_interest:
    print(t[0].split('/')[-1][:4])
    if t[0].split('/')[-1][:4] == time:
        directories.append(t[0])

a1 1


NameError: name 'time' is not defined

In [8]:
for d in directories:

    #get the number key to be replaced with a well name
    orig_name, replace_key = d.split('/')[-1].split('_')
    print(orig_name, replace_key)
    
    #get the dilution factor part
    dil_num = orig_name.split(' ')[-1]
    print(dil_num)
    
    #break off the end of the directory, so we can rename that part
    parent = d.split('/')[:-1]
    #rebuild the parent
    parent_tree = ''
    for p in parent:
        parent_tree = parent_tree + p + '/'
    
    #make the new name
    new_file = parent_tree + rename[replace_key] + ' ' + time + ' ' + dil_num
    
    print(new_file)
    
#     os.rename(d, new_file)

NameError: name 'directories' is not defined

# rename all the 'composite.tif' files with descriptive names so they can be put together for later, and so their information shows up in title bar when open in imageJ

In [10]:
#refresh tuples of interest after changes above

tuples_of_interest = get_tuples_of_interest(directory)

In [11]:
len(tuples_of_interest)

216

In [20]:
for folder in tuples_of_interest:

    #get the directory path
    dir_path = folder[0]

    #get the new name you want for the composite file
    dirname = dir_path.split('/')[-1]
    
    print(dirname)

    if ' ' in dirname:
        name_elements = dirname.split(' ')
        
    well = name_elements[0]
    
    dil = name_elements[1].rstrip('_1').split('to')
    dil = dil[0] + '^' + dil[1]
    

    filename = '/composite.tif'

    #the existing file to rename
    orig_path = dir_path + filename

    #its new name
    new_path = dir_path + '/{}_{}_merge.tif'.format(well, dil)
    
    print(new_path)

    #some folders dont have standardized data and shouldn't be used, exclude them
    try:
        os.rename(orig_path, new_path)
    except FileNotFoundError:
        print('no composite.tif found in folder {}'.format(well_time))

a1 10to2_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/a1 10to2_1/a1_10^2_merge.tif
a1 10to3_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/a1 10to3_1/a1_10^3_merge.tif
a1 10to4_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/a1 10to4_1/a1_10^4_merge.tif
a1 10to5_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/a1 10to5_1/a1_10^5_merge.tif
a10 10to2_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/a10 10to2_1/a10_10^2_merge.tif
a10 10to3_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/a10 10to3_1/a10_10^3_merge.tif
a10 10to4_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/a10 10to4_1/a10_10^4_merge.tif
a10 10to5_1

b3 10to5_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/b3 10to5_1/b3_10^5_merge.tif
b4 10to4_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/b4 10to4_1/b4_10^4_merge.tif
b4 10to5_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/b4 10to5_1/b4_10^5_merge.tif
b5 10to4_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/b5 10to4_1/b5_10^4_merge.tif
b5 10to5_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/b5 10to5_1/b5_10^5_merge.tif
b6 10to4_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/b6 10to4_1/b6_10^4_merge.tif
b6 10to5_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/b6 10to5_1/b6_10^5_merge.tif
b7 10to4_1
Z:/Reed/P

d9 10to5_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/d9 10to5_1/d9_10^5_merge.tif
e1 10to4_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/e1 10to4_1/e1_10^4_merge.tif
e1 10to5_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/e1 10to5_1/e1_10^5_merge.tif
e10 10to4_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/e10 10to4_1/e10_10^4_merge.tif
e10 10to5_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/e10 10to5_1/e10_10^5_merge.tif
e11 10to4_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/e11 10to4_1/e11_10^4_merge.tif
e11 10to5_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/e11 10to5_1/e11_10^5_merge.tif
e12 10to

g3 10to4_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/g3 10to4_1/g3_10^4_merge.tif
g3 10to5_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/g3 10to5_1/g3_10^5_merge.tif
g4 10to4_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/g4 10to4_1/g4_10^4_merge.tif
g4 10to5_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/g4 10to5_1/g4_10^5_merge.tif
g5 10to4_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/g5 10to4_1/g5_10^4_merge.tif
g5 10to5_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/g5 10to5_1/g5_10^5_merge.tif
g6 10to4_1
Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190910 a=b manual dil degorig 1/images/g6 10to4_1/g6_10^4_merge.tif
g6 10to5_1
Z:/Reed/P

# remove the merged renamed images from their subdirectories into their parent timepoint directories

In [21]:
#refresh tuples of interest after changes above

tuples_of_interest = get_tuples_of_interest(directory)

In [22]:
for t in tuples_of_interest:
    tree = t[0]
    
    files = t[2]
    
    m = [f for f in files if '_merge' in f]
    
    
    try:
        orig_name = tree + '/' + m[0]

        ##make the new name, one directory up from orig
        #break off the end of the directory, so we can rename that part
        break_off = tree.split('/')[:-1]
        #rebuild the parent
        one_up = ''
        for p in break_off:
            one_up = one_up + p + '/'

        new_name = one_up + m[0]

        os.rename(orig_name, new_name)
    
    except IndexError:
        print('could not do it for {}'.format(tree))